In [1]:
%matplotlib widget
# Import libraries
import rospy
import actionlib
from assignment_2_2024.msg import PlanningAction, PlanningGoal, vel_pos
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from actionlib_msgs.msg import GoalStatusArray
import numpy as np

In [2]:
# Initialize ROS node
rospy.init_node("notebook_controller", anonymous=True)

# Action client setup
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()

True

In [3]:
# Widgets for target input and buttons
x_input = widgets.FloatText(description='X:')
y_input = widgets.FloatText(description='Y:')
send_button = widgets.Button(description='Send Goal', button_style='success')
cancel_button = widgets.Button(description='Cancel Goal', button_style='danger')
output = widgets.Output()

# Display inputs and buttons
display(widgets.HBox([x_input, y_input]))
display(widgets.HBox([send_button, cancel_button]))
display(output)

Output()

Received position: 2.1008739471435547 3.1984429359436035
Received position: 2.1006369590759277 3.198073387145996


In [4]:
# Define button callbacks
def send_goal(b):
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x_input.value
    goal.target_pose.pose.position.y = y_input.value
    client.send_goal(goal)
    with output:
        clear_output()
        print(f"Goal sent: x={x_input.value}, y={y_input.value}")

def cancel_goal(b):
    client.cancel_goal()
    with output:
        clear_output()
        print("Goal canceled.")

send_button.on_click(send_goal)
cancel_button.on_click(cancel_goal)

In [5]:
# Robot state display widgets
position_label = widgets.Label(value="Position: x=0.00, y=0.00")
obstacle_label = widgets.Label(value="Closest Obstacle: -- m")
display(position_label)
display(obstacle_label)

Label(value='Position: x=0.00, y=0.00')

Label(value='Closest Obstacle: -- m')

In [6]:
fig, ax = plt.subplots()
line, = ax.plot([], [], 'b-')  # Line object to update

ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.set_title("Robot Trajectory")
ax.set_xlabel("x")
ax.set_ylabel("y")

x_history = []
y_history = []

def init_plot():
    line.set_data([], [])
    return line,

def update_plot(frame):
    line.set_data(x_history, y_history)
    return line,


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Counters for goal results
reached_count = 0
not_reached_count = 0

# Create another figure for the bar chart
fig2, ax2 = plt.subplots()
bars = ax2.bar(["Reached", "Not Reached"], [reached_count, not_reached_count], color=["green", "red"])
ax2.set_ylim(0, 10)
ax2.set_title("Goal Status")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Goal Status')

In [8]:
# Subscriber callbacks
def position_callback(msg):
    print("Received position:", msg.cor_x, msg.cor_y)  # Debugging
    
    position_label.value = (
        f"Position: x={msg.cor_x:.2f}, y={msg.cor_y:.2f}, "
        f"vx={msg.vel_x:.2f}, vy={msg.vel_y:.2f}"
    )
     # Append data
    x_history.append(msg.cor_x)
    y_history.append(msg.cor_y)

    # Update plot
    line.set_data(x_history, y_history)
    ax.relim()
    ax.autoscale_view()

    fig.canvas.draw_idle() 
def laser_callback(msg):
    valid = [r for r in msg.ranges if not np.isinf(r) and not np.isnan(r)]
    if valid:
        obstacle_label.value = f"Closest Obstacle: {min(valid):.2f} m"
    else:
        obstacle_label.value = "Closest Obstacle: No valid data"
        
def status_callback(msg):
    global reached_count, not_reached_count
    if not msg.status_list:
        return

    latest_status = msg.status_list[-1].status

    if latest_status == 3:  # SUCCEEDED
        reached_count += 1
    elif latest_status in [2, 4]:  # CANCELED or ABORTED
        not_reached_count += 1
    else:
        return  # Still processing

    # Update the bar chart
    bars[0].set_height(reached_count)
    bars[1].set_height(not_reached_count)
    ax2.set_ylim(0, max(reached_count, not_reached_count, 1) + 1)
    fig2.canvas.draw_idle()


In [9]:
rospy.Subscriber("/vel_pos", vel_pos, position_callback)
rospy.Subscriber("/scan", LaserScan, laser_callback)
rospy.Subscriber("/reaching_goal/status", GoalStatusArray, status_callback)

Received position: 2.1071529388427734 3.2081096172332764


In [10]:
ani = FuncAnimation(fig, update_plot, init_func=init_plot, blit=True, interval=200)
